In [1]:
import pandas as pd 

In [2]:
df=pd.read_csv("data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df=df.drop(labels=['id'],axis=1)

In [4]:
##Independent and Dependent features 
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [5]:
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude="object").columns

In [6]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [7]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [8]:
##Defining the custom ranking for ordinal categories

cut_categories=["Fair","Good","Very Good","Premium","Ideal"]
color_categories=["D","E","F","G","H","I","J"]
clarity_categories=["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]

In [9]:
from sklearn.impute import SimpleImputer #Hadndling Missing Values 
from sklearn.preprocessing import StandardScaler ## Handling Feature Scaling 
from sklearn.preprocessing import OrdinalEncoder ##Handlig Ordinal Encoder 

##pipeline 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [10]:
num_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
     ('scaler',StandardScaler())
])

#Categorical PIpeline

cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

In [11]:
##final output 

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [12]:
##Train test split 
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=40)

In [13]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())


In [14]:
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.454106,-0.296545,-0.637842,0.636818,0.663672,0.622410,0.871502,-0.317026,-0.650314
1,3.027661,-1.589990,1.965484,2.494366,2.433180,2.182754,-0.133146,0.915044,-1.315348
2,0.540612,1.458846,-1.158507,0.672887,0.645523,0.811984,-1.137795,0.915044,-1.315348
3,0.562238,-2.513880,1.444819,0.817163,0.790714,0.549496,-1.137795,-0.317026,-1.315348
4,-0.843485,-0.019378,-0.637842,-0.851023,-0.888051,-0.865022,0.871502,1.531079,1.344786


In [16]:
##Model Training 

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [22]:
regression=LinearRegression()

In [23]:
regression

LinearRegression()

In [30]:
regression.fit(X_test,y_test)

LinearRegression()

In [33]:
regression.coef_

array([[ 6399.76312109,  -134.86241889,   -70.6799942 , -1474.57386358,
         -751.79761656,   -29.67991005,    63.8803252 ,  -467.12037887,
          655.02048408]])

In [31]:
regression.intercept_

array([3961.79354377])

In [26]:
import numpy as np 
def evaluate_model(true,predicted):
    #square_error=mean_squared_error(true,predicted)
    mae=mean_absolute_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_squares=r2_score(true,predicted)
    return mae,rmse,r2_squares

In [27]:
###Multiple model training

models={ "LinearRegression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "ElasticNet":ElasticNet()}

model_list=[]
r2_mark=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #make predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_squares=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])


    print("Model Training Performance")
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2_Score:",r2_squares*100)

    r2_mark.append(r2_squares)

    print("="*40)
    print("\n")


LinearRegression
Model Training Performance
RMSE: 1017.1722564866152
MAE: 674.6638217377892
R2_Score: 93.63431403063537


Lasso
Model Training Performance
RMSE: 1015.6056722113336
MAE: 675.3910275839803
R2_Score: 93.65390698366912


Ridge
Model Training Performance
RMSE: 1017.1731135131827
MAE: 674.6868170797306
R2_Score: 93.63430330371226


ElasticNet
Model Training Performance
RMSE: 1536.886461343751
MAE: 1065.5716610505156
R2_Score: 85.46751887552743




In [34]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

In [35]:
r2_mark

[0.9363431403063537,
 0.9365390698366911,
 0.9363430330371226,
 0.8546751887552744]